# 5.4 自定义层
- **目录**
  - 5.4.1 不带参数的层
  - 5.4.2 带参数的层


- 深度学习成功背后的一个因素是神经网络的灵活性：
    - 我们可以用创造性的方式组合不同的层，从而设计出适用于各种任务的架构。
- 例如，研究人员发明了专门用于处理图像、文本、序列数据和执行动态规划的层。
- 在实际工作中，可能需要定制一个现在在深度学习框架中还不存在的层。

## 5.4.1 不带参数的层

- 首先，构造一个没有任何参数的自定义层。
- 下面的`CenteredLayer`类要从其输入中减去均值。
- 要构建它，只需继承基础层类并实现前向传播功能。

In [1]:
%matplotlib inline
import torch
import torch.nn.functional as F
from torch import nn

## 至少要实现__init__和forward两个函数
class CenteredLayer(nn.Module):
    def __init__(self):
        super().__init__()

    def forward(self, X):
        return X - X.mean()

- 让我们向该层提供一些数据，验证它是否能按预期工作。


In [2]:
layer = CenteredLayer()
layer(torch.FloatTensor([1, 2, 3, 4, 5]))

tensor([-2., -1.,  0.,  1.,  2.])

- 将层作为组件合并到更复杂的模型中。

In [3]:
net = nn.Sequential(nn.Linear(8, 128), CenteredLayer())

- 作为额外的健全性检查，可以在向该网络发送随机数据后，检查均值是否为0。
- 由于处理的是浮点数，因为存储精度的原因，仍然可能会看到一个非常小的非零数。

In [4]:
Y = net(torch.rand(4, 8))
Y.mean()

tensor(9.3132e-09, grad_fn=<MeanBackward0>)

In [9]:
net[0].weight.shape

torch.Size([128, 8])

## 5.4.2 带参数的层 

- 下面定义具有参数的层，这些参数可以通过训练进行调整。
- 可以使用内置函数来创建参数，这些函数提供一些基本的管理功能：
    - 比如管理访问、初始化、共享、保存和加载模型参数。
    - 这样做的益处在于：无需为每个自定义层编写自定义的序列化程序。
- 现在实现自定义版本的全连接层：
    - 该层需要两个参数，一个用于表示权重，另一个用于表示偏置项。
    - 我们使用修正线性单元作为激活函数。
    - 该层需要输入参数：`in_units`和`units`，分别表示输入数和输出数。


In [4]:
class MyLinear(nn.Module):
    def __init__(self, in_units, units):
        super().__init__()
        self.weight = nn.Parameter(torch.randn(in_units, units))
        self.bias = nn.Parameter(torch.randn(units,))
    def forward(self, X):
        linear = torch.matmul(X, self.weight.data) + self.bias.data
        return F.relu(linear)

- 接下来实例化`MyLinear`类并访问其模型参数。


In [5]:
linear = MyLinear(5, 3)
linear.weight

Parameter containing:
tensor([[ 0.0666,  0.2771,  0.1515],
        [ 0.1590,  0.7430,  1.5069],
        [-0.7607,  1.7223, -0.9122],
        [-0.0744, -0.3969, -0.2852],
        [-0.2314, -0.3929,  0.2304]], requires_grad=True)

- 可以使用自定义层直接执行前向传播计算。


In [6]:
## 自定义层的输出
linear(torch.rand(2, 5))

tensor([[0.0000, 0.0000, 0.8579],
        [0.0000, 0.0000, 0.2400]])

- 还可以**使用自定义层构建模型**，就像使用内置的全连接层一样使用自定义层。


In [7]:
net = nn.Sequential(MyLinear(64, 8), MyLinear(8, 1))
net(torch.rand(2, 64))

tensor([[6.4563],
        [6.3347]])

## 小结

* 我们可以通过基本层类设计自定义层。这允许我们定义灵活的新层，其行为与深度学习框架中的任何现有层不同。
* 在自定义层定义完成后，我们就可以在任意环境和网络架构中调用该自定义层。
* 层可以有局部参数，这些参数可以通过内置函数创建。

